# Manual Postprocessing - Datos Enriquecidos

Este notebook procesa los datos enriquecidos para:
1. Agrupar entidades por tipo (ciudades, itinerarios, etc.)
2. Normalizar nombres de entidades
3. Combinar entidades similares

In [56]:
import json
import pandas as pd
from collections import defaultdict
from typing import Dict, List, Set
import re

## 1. Cargar Datos Enriquecidos

In [57]:
# Cargar datos enriquecidos
input_file = "../../data/scraper_enrichment/enriched_data.jsonl"

data = []
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

print(f"Datos cargados: {len(data)} registros")
print(f"Columnas disponibles: {list(data[0].keys()) if data else 'No hay datos'}")

Datos cargados: 32 registros
Columnas disponibles: ['url', 'source_name', 'scraped_at', 'entities']


## 2. Extraer y Agrupar Entidades

In [58]:
import json
from collections import defaultdict
from pathlib import Path

# Ruta al archivo de entrada
input_path = Path('../../data/scraper_enrichment/enriched_data.jsonl')

# Cargar todas las entidades
all_entities = []
with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            data = json.loads(line)
            all_entities.extend(data.get('entities', []))

print(f'Total de entidades extraídas: {len(all_entities)}')

# Agrupar entidades por tipo
entidades_por_tipo = defaultdict(list)
for entidad in all_entities:
    tipo = entidad.get('entity_type', 'desconocido')
    
    # Manejar si tipo es una lista
    if isinstance(tipo, list):
        tipo = tipo[0] if tipo else 'desconocido'
    elif tipo is None:
        tipo = 'desconocido'
    
    entidades_por_tipo[tipo].append(entidad)

# Mostrar entidades agrupadas por tipo
for tipo, entidades in entidades_por_tipo.items():
    print(f'\n=== {tipo.upper()} ({len(entidades)}) ===')
    for e in entidades:
        nombre = e.get('name') or e.get('title') or e.get('advice_text') or '[Sin nombre]'
        desc = e.get('description') or e.get('context') or ''
        print(f'- {nombre}: {desc[:120]}')

Total de entidades extraídas: 1516

=== SITE (805) ===
- Tailandia: Diverse country with attractions for all tastes, known for its hospitality, delicious cuisine, and beautiful landscapes 
- Bangkok: Capital city of Thailand with a mix of modernity and tradition, known for its bustling streets, vibrant markets, and cul
- Bangkok: One of the must-see places in Thailand, offering a mix of Western infrastructure and Eastern traditions. Explore the Gra
- Chiang Mai: Known as the 'Northern Capital' of Thailand, offering a magical environment with nature, Buddhist temples, palaces, and 
- Chiang Mai: Explore the city for at least 2 days, with an option for additional excursions. Consider a tour to Chiang Rai from Chian
- Doi Suthep: One of the most beautiful temples to visit in Thailand.
- Phi Phi Islands: Famous island group in Thailand known for its beaches and clear turquoise waters. Includes Koh Phi Phi Don, Koh Phi Phi 
- Sukhothai Historical Park: Historical park located between Bangko

In [59]:
from difflib import SequenceMatcher
from itertools import combinations
import json
from collections import defaultdict
from pathlib import Path

def similarity(a, b):
    """Calcula la similitud entre dos strings (0-1)"""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def load_entities_from_jsonl(file_path):
    """Carga las entidades desde el archivo JSONL y las agrupa por tipo"""
    entidades_por_tipo = defaultdict(list)
    
    # Convertir a Path object para manejo profesional de rutas
    file_path = Path(file_path)
    
    if not file_path.exists():
        print(f"❌ Error: El archivo {file_path} no existe")
        return entidades_por_tipo
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data = json.loads(line)
                url = data.get('url', 'URL desconocida')
                source_name = data.get('source_name', 'Fuente desconocida')
                
                # Agregar información de origen a cada entidad
                for entidad in data.get('entities', []):
                    entidad['source_url'] = url
                    entidad['source_name'] = source_name
                    
                    tipo = entidad.get('entity_type', 'desconocido')
                    if isinstance(tipo, list):
                        tipo = tipo[0] if tipo else 'desconocido'
                    elif tipo is None:
                        tipo = 'desconocido'
                    
                    entidades_por_tipo[tipo].append(entidad)
    
    return entidades_por_tipo

def find_similar_entities_with_urls(entidades_por_tipo, threshold=0.7):
    """Encuentra entidades similares de TODOS los tipos y las agrupa, mostrando las URLs de origen"""
    
    # Extraer TODAS las entidades con información de origen
    todas_entidades_con_origen = []
    
    for tipo, entidades in entidades_por_tipo.items():
        for entidad in entidades:
            nombre = entidad.get('name') or entidad.get('title') or entidad.get('advice_text') or '[Sin nombre]'
            if nombre and nombre != '[Sin nombre]':
                todas_entidades_con_origen.append({
                    'nombre': nombre,
                    'tipo': tipo,
                    'url': entidad.get('source_url', 'URL desconocida'),
                    'source_name': entidad.get('source_name', 'Fuente desconocida'),
                    'entidad_completa': entidad  # Guardar la entidad completa para más info
                })
    
    if not todas_entidades_con_origen:
        print("No se encontraron entidades")
        return
    
    # Encontrar grupos de entidades similares
    grupos_similares = []
    entidades_usadas = set()
    
    for i, entidad1 in enumerate(todas_entidades_con_origen):
        if entidad1['nombre'] in entidades_usadas:
            continue
            
        grupo = [entidad1]
        entidades_usadas.add(entidad1['nombre'])
        
        # Buscar entidades similares
        for entidad2 in todas_entidades_con_origen[i+1:]:
            if entidad2['nombre'] not in entidades_usadas:
                sim = similarity(entidad1['nombre'], entidad2['nombre'])
                if sim >= threshold and threshold < 0.85:
                    grupo.append(entidad2)
                    entidades_usadas.add(entidad2['nombre'])
        
        if len(grupo) > 1:  # Solo grupos con más de una entidad
            grupos_similares.append(grupo)
    
    # Mostrar resultados ordenados alfabéticamente
    print(f"=== ENTIDADES SIMILARES (similitud >= {threshold}) ===")
    print(f"Total de entidades: {len(todas_entidades_con_origen)}")
    print(f"Grupos de entidades similares: {len(grupos_similares)}")
    
    # Mostrar grupos ordenados alfabéticamente
    for grupo in sorted(grupos_similares, key=lambda x: x[0]['nombre'].lower()):
        print(f"\n📍 {grupo[0]['nombre']} (principal) - Tipo: {grupo[0]['tipo']}")
        print(f"   📍 URL: {grupo[0]['url']}")
        print(f"   📰 Fuente: {grupo[0]['source_name']}")
        
        for entidad in grupo[1:]:
            sim = similarity(grupo[0]['nombre'], entidad['nombre'])
            print(f"   └─ {entidad['nombre']} (similitud: {sim:.2f}) - Tipo: {entidad['tipo']}")
            print(f"      �� URL: {entidad['url']}")
            print(f"      �� Fuente: {entidad['source_name']}")
    
    # Mostrar entidades únicas (sin similares)
    entidades_unicas = [e for e in todas_entidades_con_origen if e['nombre'] not in entidades_usadas or 
                       not any(e['nombre'] in [ent['nombre'] for ent in grupo] for grupo in grupos_similares if len(grupo) > 1)]
    
    if entidades_unicas:
        print(f"\n��️  ENTIDADES ÚNICAS ({len(entidades_unicas)}):")
        for entidad in sorted(entidades_unicas, key=lambda x: x['nombre'].lower()):
            print(f"   • {entidad['nombre']} - Tipo: {entidad['tipo']}")
            print(f"     �� URL: {entidad['url']}")
            print(f"     �� Fuente: {entidad['source_name']}")

# Configurar rutas de forma profesional usando pathlib
current_dir = Path.cwd()
print(f"�� Directorio actual: {current_dir}")

# Construir la ruta al archivo JSONL de forma relativa
file_path = current_dir.parent.parent / "data" / "scraper_enrichment" / "enriched_data.jsonl"
print(f"📄 Archivo objetivo: {file_path}")

# Verificar si el archivo existe
if not file_path.exists():
    print(f"❌ Error: El archivo {file_path} no existe")
    print("🔍 Buscando archivo en ubicaciones alternativas...")
    
    # Intentar rutas alternativas
    alternative_paths = [
        current_dir.parent.parent.parent / "data" / "scraper_enrichment" / "enriched_data.jsonl",
        current_dir.parent.parent.parent.parent / "scripts" / "data" / "scraper_enrichment" / "enriched_data.jsonl",
        Path("scripts/data/scraper_enrichment/enriched_data.jsonl")
    ]
    
    for alt_path in alternative_paths:
        if alt_path.exists():
            file_path = alt_path
            print(f"✅ Encontrado en: {file_path}")
            break
    else:
        print("❌ No se pudo encontrar el archivo enriched_data.jsonl")
        exit()

# Cargar datos desde el archivo JSONL
print(f"🔄 Cargando datos desde: {file_path}")
entidades_por_tipo = load_entities_from_jsonl(file_path)

# Mostrar estadísticas de entidades por tipo
print(f"\n�� ESTADÍSTICAS POR TIPO:")
for tipo, entidades in entidades_por_tipo.items():
    print(f"   • {tipo}: {len(entidades)} entidades")

# Ejecutar la función
find_similar_entities_with_urls(entidades_por_tipo, threshold=0.75)

�� Directorio actual: d:\TravelApp\Project\scripts\scraper_enrichment\manual_postprocessing
📄 Archivo objetivo: d:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data.jsonl
🔄 Cargando datos desde: d:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data.jsonl

�� ESTADÍSTICAS POR TIPO:
   • site: 805 entidades
   • transport: 41 entidades
   • itinerary: 69 entidades
   • desconocido: 387 entidades
   • tip: 119 entidades
   • event: 20 entidades
   • activity: 41 entidades
   • country: 18 entidades
   • province: 16 entidades
=== ENTIDADES SIMILARES (similitud >= 0.75) ===
Total de entidades: 1475
Grupos de entidades similares: 102

📍 7-Day Itinerary Option 1: Explore Bangkok and Northern Temples (principal) - Tipo: itinerary
   📍 URL: https://mundo-nomada.com/tailandia/que-ver-en-tailandia-en-7-dias/
   📰 Fuente: mundo-nomada.com
   └─ 7-Day Itinerary Option 3: From Bangkok to Northern Thailand (similitud: 0.78) - Tipo: itinerary
      �� URL: https://mundo-nom

## We check the normalized_entites integrity

In [60]:
import json
import ast
from pathlib import Path
from collections import defaultdict

def verificar_normalized_entities():
    """Verifica la estructura y contenido del archivo normalized_entities.py"""
    
    # Configurar rutas
    current_dir = Path.cwd()
    file_path = current_dir.parent.parent / "scraper_enrichment" / "auxs" / "normalized_entities.py"
    
    print("🔍 VERIFICACIÓN DE NORMALIZED_ENTITIES.PY")
    print("=" * 60)
    print(f"📄 Archivo: {file_path}")
    print(f" Directorio actual: {current_dir}")
    print()
    
    # Verificar si el archivo existe
    if not file_path.exists():
        print("❌ ERROR: El archivo normalized_entities.py no existe")
        return False
    
    print("✅ El archivo existe")
    print()
    
    # Leer el contenido del archivo
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        print("✅ Archivo leído correctamente")
    except Exception as e:
        print(f"❌ ERROR al leer el archivo: {e}")
        return False
    
    # Verificar si es un diccionario Python válido
    print("\n🔧 VERIFICACIÓN 1: ESTRUCTURA JSON/DICCIÓNARIO")
    print("-" * 50)
    
    try:
        # Buscar el diccionario en el archivo
        if 'NORMALIZED_ENTITIES = {' in content:
            # Extraer el diccionario
            start = content.find('NORMALIZED_ENTITIES = {')
            # Encontrar el final del diccionario (buscar el } que cierra el diccionario)
            brace_count = 0
            end = start
            
            for i, char in enumerate(content[start:], start):
                if char == '{':
                    brace_count += 1
                elif char == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        end = i + 1
                        break
            
            dict_content = content[start:end]
            
            # Limpiar el contenido para evaluarlo
            dict_content = dict_content.replace('NORMALIZED_ENTITIES = ', '')
            
            # Evaluar el diccionario
            normalized_entities = ast.literal_eval(dict_content)
            print("✅ Estructura de diccionario válida")
            print(f" Total de entradas: {len(normalized_entities)}")
            
        else:
            print("❌ ERROR: No se encontró 'NORMALIZED_ENTITIES = {' en el archivo")
            return False
            
    except Exception as e:
        print(f"❌ ERROR: El archivo no tiene una estructura de diccionario válida: {e}")
        print(f"   Línea problemática aproximada: {dict_content[:200]}...")
        return False
    
    # Verificar que todas las claves están en los valores
    print("\n🔍 VERIFICACIÓN 2: CLAVES EN VALORES")
    print("-" * 50)
    
    claves_no_en_valores = []
    claves_en_valores = []
    
    for clave, valores in normalized_entities.items():
        if isinstance(valores, list):
            if clave in valores:
                claves_en_valores.append(clave)
            else:
                claves_no_en_valores.append(clave)
        else:
            print(f"⚠️  ADVERTENCIA: '{clave}' no tiene una lista de valores")
    
    if claves_en_valores:
        print(f"✅ {len(claves_en_valores)} claves SÍ están en sus valores:")
        for clave in claves_en_valores:
            print(f"   • {clave}")
    
    if claves_no_en_valores:
        print(f"❌ {len(claves_no_en_valores)} claves NO están en sus valores:")
        for clave in claves_no_en_valores:
            print(f"   • {clave}")
    
    # Verificar valores duplicados entre diferentes claves
    print("\n🔄 VERIFICACIÓN 3: VALORES DUPLICADOS ENTRE CLAVES")
    print("-" * 50)
    
    todos_valores = []
    valores_por_clave = {}
    
    for clave, valores in normalized_entities.items():
        if isinstance(valores, list):
            valores_por_clave[clave] = valores
            todos_valores.extend(valores)
    
    # Encontrar valores que aparecen en múltiples claves
    valores_duplicados = defaultdict(list)
    for valor in set(todos_valores):
        claves_que_contienen = [clave for clave, valores in valores_por_clave.items() if valor in valores]
        if len(claves_que_contienen) > 1:
            valores_duplicados[valor] = claves_que_contienen
    
    if valores_duplicados:
        print(f"⚠️  ADVERTENCIA: {len(valores_duplicadas)} valores aparecen en múltiples claves:")
        for valor, claves in valores_duplicados.items():
            print(f"   • '{valor}' aparece en: {', '.join(claves)}")
    else:
        print("✅ No hay valores duplicados entre diferentes claves")
    
    # Verificar claves duplicadas
    print("\n🔑 VERIFICACIÓN 4: CLAVES DUPLICADAS")
    print("-" * 50)
    
    claves_lista = list(normalized_entities.keys())
    claves_duplicadas = [clave for clave in set(claves_lista) if claves_lista.count(clave) > 1]
    
    if claves_duplicadas:
        print(f"❌ ERROR: {len(claves_duplicadas)} claves están duplicadas:")
        for clave in claves_duplicadas:
            print(f"   • {clave}")
    else:
        print("✅ No hay claves duplicadas")
    
    # Resumen final
    print("\n📋 RESUMEN FINAL")
    print("=" * 60)
    
    total_entradas = len(normalized_entities)
    claves_ok = len(claves_en_valores)
    claves_problema = len(claves_no_en_valores)
    valores_duplicados_count = len(valores_duplicados)
    claves_duplicadas_count = len(claves_duplicadas)
    
    print(f" Total de entradas: {total_entradas}")
    print(f"✅ Claves correctas: {claves_ok}")
    print(f"❌ Claves con problemas: {claves_problema}")
    print(f"⚠️  Valores duplicados: {valores_duplicados_count}")
    print(f"❌ Claves duplicadas: {claves_duplicadas_count}")
    
    # Estado general
    if claves_problema == 0 and claves_duplicadas_count == 0:
        print("\n🎉 ¡ESTADO PERFECTO! El archivo está bien estructurado")
        return True
    elif claves_problema == 0:
        print("\n⚠️  ADVERTENCIA: Hay claves duplicadas pero la estructura es correcta")
        return False
    else:
        print("\n❌ PROBLEMAS DETECTADOS: Revisar el archivo")
        return False

# Ejecutar la verificación
resultado = verificar_normalized_entities()

🔍 VERIFICACIÓN DE NORMALIZED_ENTITIES.PY
📄 Archivo: d:\TravelApp\Project\scripts\scraper_enrichment\auxs\normalized_entities.py
 Directorio actual: d:\TravelApp\Project\scripts\scraper_enrichment\manual_postprocessing

✅ El archivo existe

✅ Archivo leído correctamente

🔧 VERIFICACIÓN 1: ESTRUCTURA JSON/DICCIÓNARIO
--------------------------------------------------
✅ Estructura de diccionario válida
 Total de entradas: 34

🔍 VERIFICACIÓN 2: CLAVES EN VALORES
--------------------------------------------------
✅ 34 claves SÍ están en sus valores:
   • Bangkok
   • Chiang Mai
   • Phuket
   • Koh Samui
   • Koh Phi Phi
   • Similan
   • Mae Hong Son
   • Songthaew
   • Malasia
   • Ayutthaya
   • Damnoen Saduak Floating Market
   • minivan
   • Night Bazaar
   • tuk-tuk
   • Wat Rong Khun
   • Chatuchak Market
   • Triángulo de Oro
   • Grand Palace of Bangkok
   • Chiang Rai
   • Krabi
   • Pattaya
   • Hua Hin
   • Koh Phangan
   • Koh Tao
   • Phra Nang
   • Kanchanaburi
   • Koh Chang

## We group the entities with same name

In [61]:
import json
import ast
from pathlib import Path
from collections import defaultdict, Counter

# === 1. Cargar el diccionario de normalización ===
def load_normalized_entities(py_path):
    with open(py_path, 'r', encoding='utf-8') as f:
        content = f.read()
    start = content.find('NORMALIZED_ENTITIES = {')
    if start == -1:
        raise ValueError("No se encontró NORMALIZED_ENTITIES en el archivo.")
    brace_count = 0
    end = start
    for i, char in enumerate(content[start:], start):
        if char == '{':
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0:
                end = i + 1
                break
    dict_content = content[start:end].replace('NORMALIZED_ENTITIES = ', '')
    return ast.literal_eval(dict_content)

# === 2. Cargar las entidades enriquecidas ===
def load_enriched_entities(jsonl_path):
    entities = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data = json.loads(line)
                url = data.get('url')
                for ent in data.get('entities', []):
                    ent['source_url'] = url
                    entities.append(ent)
    return entities

# === 3. Normalizar el nombre de la entidad ===
def normalize_entity_name(name, normalized_dict):
    if not name:
        return ""
    for norm, variations in normalized_dict.items():
        if name.lower() in [v.lower() for v in variations]:
            return norm
    return name

# === 4. Agrupar entidades (CORREGIDO) ===
def get_key(ent):
    # Agrupar por: tipo, subtipo, ciudad, nombre normalizado
    # PERO NO por descripción
    tipo = (ent.get('entity_type') or '').lower()
    subtipo = (ent.get('subtype') or '').lower()
    
    # Buscar ciudad en la jerarquía
    ciudad = None
    for h in ent.get('hierarchy', []):
        if (h.get('type') or '').lower() == 'city':
            ciudad = (h.get('name') or '').lower()
            break
    
    # Si no hay jerarquía, buscar en otros campos
    if not ciudad:
        ciudad = (ent.get('city') or '').lower()
    
    nombre = (ent.get('normalized_name') or '').lower()
    
    return (tipo, subtipo, ciudad, nombre)

def agrupar_entidades(entities, normalized_dict):
    agrupadas = defaultdict(list)
    for ent in entities:
        # Normalizar nombre
        original_name = ent.get('name') or ent.get('title') or ent.get('advice_text')
        ent['normalized_name'] = normalize_entity_name(original_name, normalized_dict)
        key = get_key(ent)
        agrupadas[key].append(ent)
    return agrupadas

# === 5. Generar la salida (MEJORADO) ===
def generar_salida(agrupadas, output_path):
    output = []
    for key, ents in agrupadas.items():
        if not key[3]:  # Saltar entidades sin nombre
            continue
            
        # Tomar la primera entidad como base
        base = dict(ents[0])
        base['appearances'] = len(ents)
        base['all_source_urls'] = list({e.get('source_url') for e in ents if e.get('source_url')})
        
        # COMBINAR DESCRIPCIONES (esto es lo que se combina, no se usa para distinguir)
        descripciones = []
        for ent in ents:
            desc = ent.get('description', '').strip()
            if desc and desc not in descripciones:
                descripciones.append(desc)
        
        if descripciones:
            if len(descripciones) == 1:
                base['description'] = descripciones[0]
            else:
                base['description'] = " | ".join(descripciones)
        
        # COMBINAR IMÁGENES
        todas_imagenes = []
        for ent in ents:
            imagenes = ent.get('images', [])
            if isinstance(imagenes, list):
                todas_imagenes.extend(imagenes)
        
        if todas_imagenes:
            base['images'] = list(set(todas_imagenes))  # Eliminar duplicados
        
        # COMBINAR WEBSITES OFICIALES
        websites = []
        for ent in ents:
            website = ent.get('official_website', '').strip()
            if website and website not in websites:
                websites.append(website)
        
        if websites:
            base['official_websites'] = websites
        
        output.append(base)
    
    # Guardar en JSONL
    with open(output_path, 'w', encoding='utf-8') as f:
        for ent in output:
            f.write(json.dumps(ent, ensure_ascii=False) + '\n')
    return output

# === 6. Estadísticas y prints bonitos ===
def print_stats(entities, agrupadas, output):
    print("🔎 Estadísticas del proceso de agrupación")
    print("="*60)
    print(f"Total de entidades originales: {len(entities)}")
    print(f"Total de grupos únicos tras agrupar: {len(agrupadas)}")
    print(f"Total de entidades en el archivo final: {len(output)}")
    print("\nEjemplo de grupo agrupado:")
    for k, v in agrupadas.items():
        if k[3]:  # Solo mostrar grupos con nombre
            print(f"  - Clave: {k}")
            print(f"    Nombres originales: {[e.get('name') for e in v]}")
            print(f"    Aparece en URLs: {[e.get('source_url') for e in v]}")
            print(f"    Número de apariciones: {len(v)}")
            print()
            break
    apariciones = [ent['appearances'] for ent in output]
    print(f"Media de apariciones por grupo: {sum(apariciones)/len(apariciones):.2f}")
    print(f"Máximo de apariciones en un grupo: {max(apariciones)}")
    print(f"Mínimo de apariciones en un grupo: {min(apariciones)}")
    print("="*60)

# === 7. Rutas ===
input_jsonl = Path(r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data.jsonl")
normalized_py = Path(r"D:\TravelApp\Project\scripts\scraper_enrichment\auxs\normalized_entities.py")
output_jsonl = Path(r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_2.jsonl")

# === 8. Proceso completo ===
normalized_dict = load_normalized_entities(normalized_py)
entities = load_enriched_entities(input_jsonl)
agrupadas = agrupar_entidades(entities, normalized_dict)
output = generar_salida(agrupadas, output_jsonl)
print_stats(entities, agrupadas, output)
print(f"\n✅ Archivo guardado en: {output_jsonl}")

🔎 Estadísticas del proceso de agrupación
Total de entidades originales: 1516
Total de grupos únicos tras agrupar: 1070
Total de entidades en el archivo final: 1056

Ejemplo de grupo agrupado:
  - Clave: ('site', 'country', '', 'thailand')
    Nombres originales: ['Tailandia', 'Thailand', 'Thailand', 'Tailandia', 'Tailandia', 'Thailand', 'Thailand', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Tailandia', 'Thailand', 'Thailand', 'Tailandia', 'Tailandia', 'Thailand', 'Thailand', 'Thailand', 'Thailand', 'Thailand', 'Thailand']
    Aparece en URLs: ['https://www.viajeroscallejeros.com/lugares-que-visitar-en-tailandia/', 'https://www.viajeroscallejeros.com/lugares-que-visitar-en-tailandia/', 'https://heymondo.es/blog/que-ver-tailandia/', 'https://www.mundoasiatours.com/que-ver-en-tailandia/', 'https://www.expansion.com/fueradeserie/viajes/2023/09/05/64f5b529e5fdeaa9778b45f4.html', 'https://lacosmopolilla.com/que-ver-en-tailandia/',

## We show the data

In [62]:
import json
from pathlib import Path
from collections import Counter

def imprimir_entidades_agrupadas(output_path, max_mostrar=10):
    """
    Imprime información detallada de las entidades agrupadas
    """
    print("🔍 IMPRESOR DE ENTIDADES AGRUPADAS")
    print("=" * 80)
    
    # Cargar el archivo de salida
    with open(output_path, 'r', encoding='utf-8') as f:
        entidades = [json.loads(line) for line in f if line.strip()]
    
    print(f"�� Total de entidades agrupadas: {len(entidades)}")
    print()
    
    # Estadísticas generales
    tipos_entidad = Counter([e.get('entity_type', 'sin_tipo') for e in entidades])
    print("�� DISTRIBUCIÓN POR TIPO DE ENTIDAD:")
    for tipo, count in tipos_entidad.most_common():
        print(f"   • {tipo}: {count}")
    print()
    
    # Entidades con más apariciones
    entidades_por_apariciones = sorted(entidades, key=lambda x: x.get('appearances', 0), reverse=True)
    print(f"🏆 TOP {max_mostrar} ENTIDADES CON MÁS APARICIONES:")
    print("-" * 80)
    
    for i, ent in enumerate(entidades_por_apariciones[:max_mostrar], 1):
        print(f"{i:2d}. {ent.get('normalized_name', ent.get('name', 'Sin nombre'))}")
        print(f"     Tipo: {ent.get('entity_type', 'N/A')}")
        print(f"     Subtipo: {ent.get('subtype', 'N/A')}")
        print(f"     Apariciones: {ent.get('appearances', 0)}")
        print(f"     URLs: {len(ent.get('all_source_urls', []))}")
        if ent.get('description'):
            desc = ent['description'][:100] + "..." if len(ent['description']) > 100 else ent['description']
            print(f"     Descripción: {desc}")
        print()
    
    # Entidades únicas (solo aparecen una vez)
    entidades_unicas = [e for e in entidades if e.get('appearances', 0) == 1]
    print(f"⭐ ENTIDADES ÚNICAS (solo 1 aparición): {len(entidades_unicas)}")
    print("-" * 80)
    
    for i, ent in enumerate(entidades_unicas[:5], 1):  # Mostrar solo las primeras 5
        print(f"{i}. {ent.get('normalized_name', ent.get('name', 'Sin nombre'))} - {ent.get('entity_type', 'N/A')}")
    
    if len(entidades_unicas) > 5:
        print(f"   ... y {len(entidades_unicas) - 5} más")
    print()
    
    # Ejemplos de grupos con múltiples apariciones
    grupos_multiples = [e for e in entidades if e.get('appearances', 0) > 1]
    print(f"🔄 GRUPOS CON MÚLTIPLES APARICIONES: {len(grupos_multiples)}")
    print("-" * 80)
    
    for i, ent in enumerate(grupos_multiples[:3], 1):  # Mostrar solo los primeros 3
        print(f"{i}. {ent.get('normalized_name', ent.get('name', 'Sin nombre'))}")
        print(f"   Apariciones: {ent.get('appearances', 0)}")
        print(f"   URLs de origen:")
        for url in ent.get('all_source_urls', [])[:3]:  # Mostrar solo las primeras 3 URLs
            print(f"     • {url}")
        if len(ent.get('all_source_urls', [])) > 3:
            print(f"     • ... y {len(ent.get('all_source_urls', [])) - 3} más")
        print()
    
    # Verificar normalización
    print("🔧 VERIFICACIÓN DE NORMALIZACIÓN:")
    print("-" * 80)
    
    entidades_sin_normalizar = [e for e in entidades if e.get('normalized_name') == e.get('name')]
    entidades_normalizadas = [e for e in entidades if e.get('normalized_name') != e.get('name')]
    
    print(f"   • Entidades normalizadas: {len(entidades_normalizadas)}")
    print(f"   • Entidades sin cambios: {len(entidades_sin_normalizar)}")
    
    if entidades_normalizadas:
        print("   Ejemplos de normalización:")
        for ent in entidades_normalizadas[:3]:
            print(f"     • '{ent.get('name')}' → '{ent.get('normalized_name')}'")
    print()
    
    # Resumen final
    print("📋 RESUMEN FINAL")
    print("=" * 80)
    print(f"✅ Total entidades procesadas: {len(entidades)}")
    print(f"✅ Entidades únicas: {len(entidades_unicas)}")
    print(f"✅ Grupos múltiples: {len(grupos_multiples)}")
    print(f"✅ Tipos diferentes: {len(tipos_entidad)}")
    print(f"✅ Entidades normalizadas: {len(entidades_normalizadas)}")
    
    # Calcular estadísticas de apariciones
    apariciones = [e.get('appearances', 0) for e in entidades]
    if apariciones:
        print(f"✅ Media apariciones: {sum(apariciones)/len(apariciones):.2f}")
        print(f"✅ Máximo apariciones: {max(apariciones)}")
        print(f"✅ Mínimo apariciones: {min(apariciones)}")

# Ejecutar el impresor
output_path = Path(r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_2.jsonl")
imprimir_entidades_agrupadas(output_path, max_mostrar=15)

🔍 IMPRESOR DE ENTIDADES AGRUPADAS
�� Total de entidades agrupadas: 1056

�� DISTRIBUCIÓN POR TIPO DE ENTIDAD:
   • site: 522
   • sin_tipo: 236
   • tip: 119
   • itinerary: 58
   • activity: 41
   • transport: 27
   • event: 19
   • country: 18
   • province: 16

🏆 TOP 15 ENTIDADES CON MÁS APARICIONES:
--------------------------------------------------------------------------------
 1. Thailand
     Tipo: site
     Subtipo: country
     Apariciones: 26
     URLs: 19
     Descripción: Diverse country with attractions for all tastes, known for its hospitality, delicious cuisine, and b...

 2. Chiang Mai
     Tipo: site
     Subtipo: city
     Apariciones: 17
     URLs: 12
     Descripción: Known as the 'Northern Capital' of Thailand, offering a magical environment with nature, Buddhist te...

 3. Phuket
     Tipo: site
     Subtipo: island
     Apariciones: 16
     URLs: 14
     Descripción: Popular island destination in Thailand. | Phuket is the largest island in Thailand, known for it

## We do manual modification of the entities

In [67]:
# === BLOQUE PARA JUPYTER NOTEBOOK ===
import sys
from pathlib import Path

# Añadir el directorio del script al path
current_dir = Path.cwd()
script_dir = current_dir / "entidades_editor.py"
sys.path.append(str(current_dir))

# Importar la clase
from entidades_editor import iniciar_editor

# Definir rutas
input_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl"
output_path = r"D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl"

print("🚀 Iniciando Editor de Entidades...")
print(f"📥 Entrada: {input_path}")
print(f"📤 Salida: {output_path}")
print()

# Iniciar el editor
iniciar_editor(input_path, output_path)

🚀 Iniciando Editor de Entidades...
📥 Entrada: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
📤 Salida: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl

✅ Usando archivo existente: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Cargadas 1051 entidades desde D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ OSM añadido: OSM:N2082718893
✅ Wikidata añadido: Q669271
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardadas en: D:\TravelApp\Project\scripts\data\scraper_enrichment\enriched_data_3.jsonl
✅ Entidades guardad